# Рубежный контроль №1. 
**"Монотонный итеративный алгоритм решения матричной игры"**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание
Пусть с помощью матрицы А задана матричная антагонистическая игра двух игроков. По данной матрице необходимо найти решение игры для первого игрока (то есть, вектор оптимальных стратегий и цену игры), используя монотонный итеративный алгоритм.

Игра Г размера (3 x 3) задана матрицей:
 
| 1  | 11 | 11 |
|:--:|:--:|:--:|
| 7  | 5  | 8  | 
| 16 | 6  | 2  | 


In [1]:
import logging
from pathlib import Path

import numpy as np

from game_theory.utils.matrix_games.monotonic.monotonic import Monotonic

from game_theory.utils.matrix_games.game_matrix import GameMatrix
from game_theory.utils.matrix_games.mixed_strategies import (
    check_resulted_game_price,
    get_resulted_mixed_strategies,
)

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [2]:
# Входная матрица прямоугольной игры с нулевой суммой.
game_matrix = GameMatrix(
    matrix=np.array(
        [
            [1, 11, 11],
            [7, 5, 8],
            [16, 6, 2],
        ],
        # [
        #     [19, 7, 3],
        #     [6, 9, 9],
        #     [8, 2, 11],
        # ],
        dtype=int,
    )
)
game_matrix

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

In [3]:
print(f"Нижняя цена игры:  {game_matrix.lowest_game_price[1]}\n"
      f"Верхняя цена игры: {game_matrix.highest_game_price[1]}")

Нижняя цена игры:  5
Верхняя цена игры: 11


In [4]:
reduced_game: GameMatrix = game_matrix.reduce_dimension(method='dominant_absorption')
reduced_game

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

### Монотонный итеративный алгоритм

In [5]:
reduced_game

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

In [6]:
monotonic_method = Monotonic(reduced_game)

In [7]:
monotonic_method.solve()

Решение игры относительно игрока A
Итерация 0:
x^0 = [1 0 0]
c^0 = [ 1 11 11]
v^0 = 1
J^0 = [0]
Итерация 1:
Рассмотрим подыгру Г^1: 
[[ 1]
 [ 7]
 [16]]
Седловая точка найдена: (16, [0, 0, 1])
Оптимальная стратегия игрока: 
	‾x_1 = (0, 0, 1)
	‾c_1 = (16, 6, 2)
Находим оптимальную стратегию игрока в подыгре из двух строк: 
[[ 1 11 11]
 [16  6  2]]
Удаление NBR-стратегий ['b2']
Матрица после уменьшения размерности: 
[[ 1 11]
 [16  2]]
F = b^T⋅x -> min,
A^T⋅x >= c^T,
x1, x2, ..., xn >= 0
b^T = [1 1]
A^T =
[[ 1 16]
 [11  2]],
c^T = [1 1].
Процесс решения:
Поиск опорного решения: 
Исходная симплекс-таблица:
+----+---------+----------+----------+
|    |   Si0   |    x1    |    x2    |
+----+---------+----------+----------+
| x3 | -1.0000 | -1.0000  | -16.0000 |
| x4 | -1.0000 | -11.0000 | -2.0000  |
| F  |  0.0000 | -1.0000  | -1.0000  |
+----+---------+----------+----------+
Разрешающая строка: x4
Разрешающий столбец: x1
+----+---------+---------+----------+
|    |   Si0   |    x4   |    x2 

((7.249999999999999, array([0.58333333, 0.        , 0.41666667])),
 (7.25, array([0.375, 0.   , 0.625])))